In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [74]:
df = pd.read_excel('ready_to_classify.xlsx')
df.head()

,Unnamed: 0,Carimbo de data/hora,id_integer,role,years_exp,org_size,use_metrics_planning,use_metrics_review,use_metrics_weekly,use_metrics_daily,use_metrics_retro,metrics_category,agile_methods_filtered,matched_metrics,not_matched_metrics
0,0,17:34:49,1,Product manager,0 a 5,Microempresa,Métricas,0,Métricas,0,0,"Cronograma e progresso, Produto, Tecnologia, C...","['Scrum', 'Kanban']",['ideal team capacity'],[]
1,1,17:57:23,2,Desenvolvedor(a),0 a 5,Microempresa,Métricas,Métricas,0,0,0,"Cronograma e progresso, Produto, Cliente","['Scrum', 'Kanban']","['nps', 'burndown', 'outstanding bugs', 'throu...",[]
2,2,20:50:58,3,Product manager,0 a 5,Pequena empresa,NaN,Métricas,0,NaN,Métricas,"Cronograma e progresso, Pessoas, Produto",['Scrum'],"['burndown', 'velocity']","['okr', 'quantidade de solicitações de clientes']"
3,3,22:26:06,4,Team leader,6 a 9,Microempresa,Métricas,NaN,NaN,NaN,Métricas,"Cronograma e progresso, Produto, Cliente","['Scrum', 'Kanban']",['throughput'],"['cac', 'roadmaps', 'conversão']"
4,4,19:07:11,5,Product manager,0 a 5,Pequena empresa,Métricas,Métricas,0,0,Métricas,Cronograma e progresso,"['Scrum', 'Kanban']","['user story points', 'nps']",[]


In [75]:
# Simplificar o dataset removendo colunas desnecessárias
df_cleaned = df[['role', 'years_exp', 'org_size', 'use_metrics_planning',
                 'use_metrics_review', 'use_metrics_weekly', 'use_metrics_daily',
                 'use_metrics_retro', 'agile_methods_filtered', 'metrics_category']]


In [76]:
# Definir as categorias únicas para classificação multilabel
categories = ['Cronograma e progresso', 'Produto', 'Processo', 'Tecnologia', 'Pessoas', 'Cliente']

# Criar colunas binárias para cada categoria
for category in categories:
    df_cleaned[category] = df_cleaned['metrics_category'].apply(lambda x: 1 if category in x else 0)

/var/folders/5j/0g6pzkwj7_g1p9xwqx5jfgq80000gn/T/ipykernel_63951/2031624344.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[category] = df_cleaned['metrics_category'].apply(lambda x: 1 if category in x else 0)
/var/folders/5j/0g6pzkwj7_g1p9xwqx5jfgq80000gn/T/ipykernel_63951/2031624344.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[category] = df_cleaned['metrics_category'].apply(lambda x: 1 if category in x else 0)
/var/folders/5j/0g6pzkwj7_g1p9xwqx5jfgq80000gn/T/ipykernel_639

In [77]:
# Explodir o dataframe para criar uma linha para cada categoria
exploded_data = []

# Loop por cada linha do dataframe
for idx, row in df_cleaned.iterrows():
    # Obter as categorias associadas a essa instância
    categories_present = [category for category in categories if row[category] == 1]
    
    # Criar uma nova linha para cada categoria presente
    for category in categories_present:
        new_row = row.copy()
        # Definir todas as categorias como 0, exceto a atual
        for cat in categories:
            new_row[cat] = 1 if cat == category else 0
        exploded_data.append(new_row)

# Converter a lista de linhas em um novo dataframe
df_exploded = pd.DataFrame(exploded_data)

In [78]:
# One-hot encoding para variáveis categóricas
df_exploded_encoded = pd.get_dummies(df_exploded, columns=['role', 'years_exp', 'org_size'])


In [79]:
# Tratar as colunas relacionadas ao uso de métricas
metrics_columns = ['use_metrics_planning', 'use_metrics_review', 'use_metrics_weekly', 'use_metrics_daily', 'use_metrics_retro']

# Atualizar as colunas para considerar 1 sempre que houver algum valor presente, e 0 caso contrário
for col in metrics_columns:
    df_exploded_encoded[col] = df_exploded[col].apply(lambda x: 1 if x != 0 and pd.notna(x) else 0)

In [80]:
import ast

# Verificar se o conteúdo da coluna 'agile_methods_filtered' está no formato correto
def method_present(method, method_list_str):
    # Verificar se a string está no formato de lista e converter para uma lista real
    try:
        method_list = ast.literal_eval(method_list_str)  # Converte a string da lista em uma lista Python
    except (ValueError, SyntaxError):
        method_list = []  # Se a conversão falhar, retornar uma lista vazia
    return 1 if method in method_list else 0

# Expandir a coluna 'agile_methods_filtered' para colunas binárias
agile_methods = ['Scrum', 'Kanban', 'Lean', 'XP', 'Safe', 'ScrumBan']

# Para cada método ágil, criar uma nova coluna binária
for method in agile_methods:
    df_exploded_encoded[method] = df_exploded['agile_methods_filtered'].apply(lambda x: method_present(method, x))

# Excluir a coluna 'agile_methods_filtered' após a conversão
df_exploded_encoded = df_exploded_encoded.drop(columns=['agile_methods_filtered'])


In [81]:
# Excluir a coluna 'metrics_category' para evitar o erro com string
df_exploded_encoded = df_exploded_encoded.drop(columns=['metrics_category'])

In [82]:
df_exploded_encoded.to_excel('df_exploded_encoded.xlsx')

In [84]:
# Dividir os dados em conjunto de treino (60%), validação (20%) e teste (20%)
X = df_exploded_encoded.drop(columns=categories)  # Features
classification_reports = {}  # Dicionário para armazenar relatório

In [85]:
from sklearn.metrics import classification_report

In [87]:
for category in categories:
    y = df_exploded_encoded[category]  # Target (classificação binária)
    
    # Dividir os dados em treino+validação (80%) e teste (20%)
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Dividir treino+validação em treino (60%) e validação (20%)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

    # Inicializar e treinar o modelo
    rf_binary = RandomForestClassifier(random_state=42)
    rf_binary.fit(X_train, y_train)
    
    # Avaliar o desempenho no conjunto de validação
    y_pred_val = rf_binary.predict(X_val)
    
    # Avaliar o modelo no conjunto de teste
    y_pred_test = rf_binary.predict(X_test)
    
    # Gerar o relatório de classificação para o conjunto de teste
    report = classification_report(y_test, y_pred_test, output_dict=True)
    
    # Armazenar o relatório para esta categoria
    classification_reports[category] = report

# Converter os relatórios de classificação em um DataFrame para visualização
classification_reports_df = pd.DataFrame.from_dict(classification_reports)

display(classification_reports_df)

,Cronograma e progresso,Produto,Processo,Tecnologia,Pessoas,Cliente
0,"{'precision': 0.7457627118644068, 'recall': 0....","{'precision': 0.7637795275590551, 'recall': 0....","{'precision': 0.8625954198473282, 'recall': 0....","{'precision': 0.8106060606060606, 'recall': 0....","{'precision': 0.859504132231405, 'recall': 0.8...","{'precision': 0.875968992248062, 'recall': 0.9..."
1,"{'precision': 0.05263157894736842, 'recall': 0...","{'precision': 0.0, 'recall': 0.0, 'f1-score': ...","{'precision': 0.0, 'recall': 0.0, 'f1-score': ...","{'precision': 0.0, 'recall': 0.0, 'f1-score': ...","{'precision': 0.0, 'recall': 0.0, 'f1-score': ...","{'precision': 0.0, 'recall': 0.0, 'f1-score': ..."
accuracy,0.649635,0.708029,0.824818,0.781022,0.759124,0.824818
macro avg,"{'precision': 0.3991971454058876, 'recall': 0....","{'precision': 0.38188976377952755, 'recall': 0...","{'precision': 0.4312977099236641, 'recall': 0....","{'precision': 0.4053030303030303, 'recall': 0....","{'precision': 0.4297520661157025, 'recall': 0....","{'precision': 0.437984496124031, 'recall': 0.4..."
weighted avg,"{'precision': 0.58892282047442, 'recall': 0.64...","{'precision': 0.5965285361227656, 'recall': 0....","{'precision': 0.749261715049869, 'recall': 0.8...","{'precision': 0.6626852466268525, 'recall': 0....","{'precision': 0.7528503348012306, 'recall': 0....","{'precision': 0.7736660442482883, 'recall': 0...."
